In [155]:
# Load the model
from keras.models import load_model
import os
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint

In [156]:
model_file = os.path.join('..', 'model', 'grammy_prediction_model.h5')

grammy_prediction_model = load_model(model_file)

- 'Year', 
- 'Acousticness', 
- 'Danceability', 
- 'Duration (ms)', 
- 'Energy',
- 'Instrumentalness', 
- 'Key', 
- 'Liveness', 
- 'Loudness', 
- 'Mode',
- 'Speechiness',
- 'Tempo',
- 'Time Signature',
- 'Valence',
- 'Explicit',
- 'Popularity'

In [157]:
# Spotify API Keys
spotify_cliend_id='bca78196e824433fbdf88ec18d84825f'
spotify_client_secret='d43763215bd8435eb9b3faaf048ca038'
os.environ['SPOTIPY_CLIENT_ID']=spotify_cliend_id
os.environ['SPOTIPY_CLIENT_SECRET']=spotify_client_secret

In [158]:
# Generate API Token
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [159]:
def get_features(title, artist):
    results = sp.search(q=f'track:{title} artist:{artist}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'{title}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        return
    track_info = results['tracks']['items'][0]
    track_id = track_info['id']
    features = sp.audio_features([track_id])[0]
    
    if features is None:
        return
    
    #pprint(track_info)
    
    features_as_dict = {
        'Year': track_info['album']['release_date'][:4],
        'Acousticness': features['acousticness'],
        'Danceability': features['danceability'],
        'Duration (ms)': features['duration_ms'],
        'Energy': features['energy'],
        'Instrumentalness': features['instrumentalness'],
        'Key': features['key'],
        'Liveness': features['liveness'],
        'Loudness': features['loudness'],
        'Mode': features['mode'],
        'Speechiness': features['speechiness'],
        'Tempo': features['tempo'],
        'Time Signature': features['time_signature'],
        'Valence': features['valence'],
        'Explicit': track_info['explicit'],
        'Popularity': track_info['popularity']

    }
    return features_as_dict

In [160]:
test_data = [
    get_features('Adele', 'Someone like you'),
    get_features('Adele', 'Hello'),
    get_features('Stressed Out', 'twenty one pilots'),
    get_features('Dystopia', 'Megadeth'),
    get_features('Despacito', 'Luis Fonsi'),
    get_features("That's What I Like", 'Bruno Mars'),
    get_features("Beat it", 'Michael Jackson'),
    get_features("Black or white", 'Michael Jackson'),
]
test_data

[{'Year': '2011',
  'Acousticness': 0.892,
  'Danceability': 0.559,
  'Duration (ms)': 285040,
  'Energy': 0.33,
  'Instrumentalness': 0,
  'Key': 9,
  'Liveness': 0.0975,
  'Loudness': -8.251,
  'Mode': 1,
  'Speechiness': 0.0285,
  'Tempo': 135.109,
  'Time Signature': 4,
  'Valence': 0.285,
  'Explicit': False,
  'Popularity': 78},
 {'Year': '2011',
  'Acousticness': 0.892,
  'Danceability': 0.559,
  'Duration (ms)': 285040,
  'Energy': 0.33,
  'Instrumentalness': 0,
  'Key': 9,
  'Liveness': 0.0975,
  'Loudness': -8.251,
  'Mode': 1,
  'Speechiness': 0.0285,
  'Tempo': 135.109,
  'Time Signature': 4,
  'Valence': 0.285,
  'Explicit': False,
  'Popularity': 78},
 {'Year': '2015',
  'Acousticness': 0.0462,
  'Danceability': 0.734,
  'Duration (ms)': 202333,
  'Energy': 0.637,
  'Instrumentalness': 2.29e-05,
  'Key': 4,
  'Liveness': 0.0602,
  'Loudness': -5.677,
  'Mode': 0,
  'Speechiness': 0.141,
  'Tempo': 169.977,
  'Time Signature': 4,
  'Valence': 0.648,
  'Explicit': False,
  

In [161]:
df = pd.DataFrame(test_data)[['Year', 'Acousticness', 'Danceability', 'Duration (ms)', 'Energy',
       'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode',
       'Speechiness', 'Tempo', 'Time Signature', 'Valence', 'Explicit',
       'Popularity']]
df.Explicit = df.Explicit.astype(int)
#excluded_features = ['Year',  'Duration (ms)', 'Key',  'Mode', 'Speechiness', 'Tempo', 'Valence', 'Explicit']
excluded_features = ['Popularity']
df.drop(labels=excluded_features, axis=1, inplace=True)
df

,Year,Acousticness,Danceability,Duration (ms),Energy,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time Signature,Valence,Explicit
0,2011,0.89200,0.559,285040,0.330,0.000000,9,0.0975,-8.251,1,0.0285,135.109,4,0.285,0
1,2011,0.89200,0.559,285040,0.330,0.000000,9,0.0975,-8.251,1,0.0285,135.109,4,0.285,0
2,2015,0.04620,0.734,202333,0.637,0.000023,4,0.0602,-5.677,0,0.1410,169.977,4,0.648,0
3,2016,0.00006,0.274,299107,0.989,0.019300,10,0.0791,-2.895,0,0.4830,109.985,4,0.148,0
4,2017,0.22800,0.653,228827,0.816,0.000000,2,0.0967,-4.353,1,0.1670,178.086,4,0.816,0
5,2017,0.01350,0.845,204773,0.569,0.000000,1,0.0898,-5.127,1,0.0414,134.047,4,0.848,0
6,1982,0.04910,0.779,258040,0.867,0.000008,3,0.1970,-3.704,0,0.0457,138.858,4,0.915,0
7,2005,0.08240,0.741,202853,0.894,0.052700,4,0.0890,-3.826,1,0.0495,114.869,4,0.960,0


In [162]:
from sklearn.externals import joblib
scaler_file = os.path.join('..', 'model', 'grammy_prediction_scaler.sav')

X_scaler = joblib.load(scaler_file) 
df_scaled = X_scaler.transform(df)

/Users/kishore/Tools/anaconda3/envs/PythonStuff/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  """


In [163]:
df_scaled

array([[ 1.25324411e+00,  1.81525796e+00, -1.19673868e-02,
         3.22761251e-01, -1.13398992e+00, -3.91221121e-01,
         1.07918862e+00, -5.75887341e-01,  2.15267975e-01,
         6.39307198e-01, -4.73973083e-01,  5.18632614e-01,
         2.22103987e-01, -9.88514329e-01, -2.87233336e-01],
       [ 1.25324411e+00,  1.81525796e+00, -1.19673868e-02,
         3.22761251e-01, -1.13398992e+00, -3.91221121e-01,
         1.07918862e+00, -5.75887341e-01,  2.15267975e-01,
         6.39307198e-01, -4.73973083e-01,  5.18632614e-01,
         2.22103987e-01, -9.88514329e-01, -2.87233336e-01],
       [ 1.48569426e+00, -8.48373962e-01,  1.01551641e+00,
        -3.51561672e-01,  1.48047128e-01, -3.91125708e-01,
        -3.39669108e-01, -7.90429354e-01,  7.44788607e-01,
        -1.56419324e+00,  7.15462342e-01,  1.72751620e+00,
         2.22103987e-01,  4.15426668e-01, -2.87233336e-01],
       [ 1.54380679e+00, -9.93680153e-01, -1.68529814e+00,
         4.37451671e-01,  1.61800492e+00, -3.10808372

In [164]:
predictions = grammy_prediction_model.predict_classes(df_scaled)
predictions

array([1, 1, 1, 0, 0, 1, 1, 0])

In [166]:
from keras.utils import to_categorical

metric_names = grammy_prediction_model.metrics_names
metric_values = grammy_prediction_model.evaluate(df_scaled, to_categorical(predictions))
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

8/8 [==============================] - 0s 43ms/step
loss :  0.04756639525294304
acc :  1.0
